In [43]:
import pandas as pd
import re

In [37]:
kw_list_df = pd.read_csv("./data/keyword_list.csv")
kw_list_df

,Group,Keywords
0,0,"coat, jacket"
1,1,blazer
2,2,"hoodies, sweater, sweatshirt"
3,3,vest
4,4,cardigan
...,...,...
276,276,"dry iron, garment steamers, steam iron, travel..."
277,277,"washing machine, clothes dryer"
278,278,"crt tv, lcd tv, led tv, oled tv, plasma tv, ql..."
279,279,"tv remote control, tv antenna, tv box"


In [51]:
kw_qn_df = pd.read_csv("./data/keyword_question.csv")
kw_qn_df

,index,name
0,0,IZRA Cotton Milky Gray Blouse + Skirt Putih
1,1,(Coral Red) IZRA BOUTIQUE Baju Kurung Moden
2,2,(Red) IZRA BOUTIQUE Baju Kurung Moden
3,3,(Peach) IZRA BOUTIQUE Baju Kurung Moden With P...
4,4,(Black) IZRA BOUTIQUE Baju Kurung Moden
...,...,...
799995,799995,iPhone X XS XR XSMAX 6 6S 7 8 PLUS Boy Girl so...
799996,799996,"melissa melicisa meli's shoes, shoes, jelly sh..."
799997,799997,plaid stitching summer o-neck short sleeve mat...
799998,799998,kimchi's shop loose ins pants bf straight shor...


In [39]:
kw_dict = dict()
for index, row in kw_list_df.iterrows():
    group = int(row["Group"])
    split_kw = row["Keywords"].lower().split(", ")
    for kw in split_kw:
        if kw not in kw_dict:
            kw_dict[kw] = 999
        if group < kw_dict[kw]:
            kw_dict[kw] = group
print(kw_dict)

{'coat': 0, 'jacket': 0, 'blazer': 1, 'hoodies': 2, 'sweater': 2, 'sweatshirt': 2, 'vest': 3, 'cardigan': 4, 'raincoat': 5, 'windbreaker': 5, 'jeans': 6, 'denim pants': 6, 'palazzo': 7, 'pants': 8, 'trousers': 8, 'sweatpants': 9, 'joggers': 9, 'trackpants': 9, 'leggings': 10, 'compression pants': 10, 'tights': 10, 'jumpsuits': 11, 'bodysuits': 11, 'overall': 11, 'playsuits': 12, 'singlet': 13, 'tank tops': 13, 'camisole': 13, 'shapewear': 14, 'corset': 14, 'panty-hose': 15, 'stocking': 15, 'panties': 16, 'brief': 17, 'shorts': 18, 'hot pants': 18, 'trunks': 19, 'boxer': 19, 'bandage bra': 20, 'sport bra': 20, 'push-up': 20, 'silicone bra': 20, 'nipple cover': 20, 'bikini': 20, 'night gown': 21, 'baju tidur': 21, 'night dress': 21, 'pyjamas': 21, 'swimsuit': 22, 'diving suit': 22, 'board shorts': 22, 'rash guard': 22, 'baju kurung': 23, 'baju kebaya': 23, 'baju kemeja': 23, 'blouse muslimah': 24, 'jubah muslimah': 24, 'telekung': 25, 'sejadah': 25, 'saree': 26, 'anarkali': 26, 'punjabi 

In [40]:
max_kw_len = 0
for k in kw_dict.keys():
    kw_len = len(k.split())
    if kw_len > max_kw_len:
        max_kw_len = kw_len
max_kw_len

5

In [75]:
def clean(text):
    text = text.lower()
    text = re.sub("[^A-Za-z0-9 \-]", "", text)
    text = text.split()
    return text

df = kw_qn_df.copy()
df["name"] = df["name"].map(clean)
df

,index,name
0,0,"[izra, cotton, milky, gray, blouse, skirt, putih]"
1,1,"[coral, red, izra, boutique, baju, kurung, moden]"
2,2,"[red, izra, boutique, baju, kurung, moden]"
3,3,"[peach, izra, boutique, baju, kurung, moden, w..."
4,4,"[black, izra, boutique, baju, kurung, moden]"
...,...,...
799995,799995,"[iphone, x, xs, xr, xsmax, 6, 6s, 7, 8, plus, ..."
799996,799996,"[melissa, melicisa, melis, shoes, shoes, jelly..."
799997,799997,"[plaid, stitching, summer, o-neck, short, slee..."
799998,799998,"[kimchis, shop, loose, ins, pants, bf, straigh..."


In [76]:
def process(text):
    groups = set()
    
    start = 0
    while start < len(text):
        max_subseq = min(max_kw_len, len(text) - start)
        for i in range(max_subseq, 0, -1):
            subseq = text[start : start + i]
            joined = " ".join(subseq)
            if joined in kw_dict:
                groups.add(kw_dict[joined])
                start += (i - 1)
                break
        start += 1
    
    groups = sorted(list(groups))
    return groups

df["groups_found"] = df["name"].map(process)
df

,index,name,groups_found
0,0,"[izra, cotton, milky, gray, blouse, skirt, putih]","[31, 33]"
1,1,"[coral, red, izra, boutique, baju, kurung, moden]",[23]
2,2,"[red, izra, boutique, baju, kurung, moden]",[23]
3,3,"[peach, izra, boutique, baju, kurung, moden, w...",[23]
4,4,"[black, izra, boutique, baju, kurung, moden]",[23]
...,...,...,...
799995,799995,"[iphone, x, xs, xr, xsmax, 6, 6s, 7, 8, plus, ...","[118, 122]"
799996,799996,"[melissa, melicisa, melis, shoes, shoes, jelly...","[45, 46]"
799997,799997,"[plaid, stitching, summer, o-neck, short, slee...",[164]
799998,799998,"[kimchis, shop, loose, ins, pants, bf, straigh...","[8, 18]"


In [77]:
output_df = df.copy()
output_df = output_df.drop("name", axis = 1)
output_df.to_csv("./data/output.csv", index = False)
output_df

,index,groups_found
0,0,"[31, 33]"
1,1,[23]
2,2,[23]
3,3,[23]
4,4,[23]
...,...,...
799995,799995,"[118, 122]"
799996,799996,"[45, 46]"
799997,799997,[164]
799998,799998,"[8, 18]"
